In [ ]:
import matplotlib.pyplot as plt
import torch as t
from m4.data.dataset_extractor import M4DatasetExtractor, M4DatasetGroup
from fcast.utils.serialization import deserialize
from m4.pipeline.forecaster.m4_baselines import theta
from fcast.pipeline.transformation.scaling import Scaling
from m4.pipeline.model.lstm import UniqueLSTM
from fcast.domain.dataset import Dataset
from m4.pipeline.adaptors import dataframe_to_tensor, default_torch_device, tensor_to_dataframe
from pandas import DataFrame

def df_to_tensor(df):
    return t.unsqueeze(dataframe_to_tensor(df), 0)

def tensor_to_df(tensor, df):
    return tensor_to_dataframe(t.squeeze(tensor, 0), df.columns)

def minmax_scale(df):
    min_v = float(df.min(axis=0))
    max_v = float(df.max(axis=0))
    denominator = max_v - min_v
    return df.apply(lambda v: (v - min_v) / denominator)


extractor = M4DatasetExtractor(dataset_path='/projects/eai-m4/dataset', group=M4DatasetGroup.daily)
train_dataset = extractor.training_set()
ts = train_dataset[1].data

input_ts = minmax_scale(ts)


print(f'original: {ts.shape}\ninput ts shape: {input_ts.shape}\ninput tensor shape: {df_to_tensor(input_ts).shape}')

### Model

In [ ]:
class TSLSTM(t.nn.Module):
    def __init__(self, name, hidden_size, num_layers, optim, loss):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.name = name
        self.optim = optim
        self.loss = loss
        
        self.lstm = t.nn.LSTM(input_size=1,
                              hidden_size=hidden_size,
                              num_layers=num_layers,
                              batch_first=True  # expects input as (batch, seq, features)
                             )
        self.output_layer = t.nn.Linear(in_features=hidden_size, out_features=1)                          
        
    def update(self, df):
        x = df_to_tensor(df)
        optim = self.optim(self.parameters())
        loss = t.nn.L1Loss(reduction='elementwise_mean')
        prediction = self(x[:, :-1, :])
        loss_fn = loss(prediction, x[:, 1:, :])
        loss_fn.backward() # calculates gradients for each model parameter (looking at prediction)
        optim.step() # updates weights
        return float(loss_fn)
                        
    def forward(self, x):
        output, _ = self.lstm(x)
        output = self.output_layer(output)
        return output
    
    def predict(self, df):
        return tensor_to_df(self(df_to_tensor(df)), df)


### Training

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
from IPython import display

def train(models):
    losses = dict([(model.name, []) for model in models])
    predictions = dict([(model.name, []) for model in models])
    
    while True:      
        for model in models:
            losses[model.name] += [model.update(input_ts)]
            predictions[model.name] = model.predict(input_ts)                
        
        plt.figure(num=1, figsize=(16, 8), dpi=120)
        plt.clf()
        
        plt.plot(input_ts, label='original')
        for model in models:
            plt.plot(predictions[model.name], label=model.name)
        plt.legend()        
        f1 = plt.gcf()

        plt.figure(num=2, figsize=(16, 8), dpi=120)
        plt.clf()
        for model in models:            
            plt.plot(losses[model.name], label=model.name)
        plt.legend()
        f2 = plt.gcf()
        
        f1.canvas.draw()                    
        f2.canvas.draw()
        
        

In [ ]:
sgd = TSLSTM('sgd', 100, 1, lambda p: t.optim.SGD(p, lr=0.001), t.nn.L1Loss(reduction='elementwise_mean'))
adam = TSLSTM('adam', 100, 1, lambda p: t.optim.Adam(p), t.nn.L1Loss(reduction='elementwise_mean'))

train([adam, sgd])